In [ ]:
import time
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import talib

# === CONFIG ===
SYMBOLS = [
    "XAUUSD", "EURUSD", "GBPUSD", "USDJPY", "AUDUSD",
    "USDCAD", "NZDUSD", "USDCHF", "EURJPY", "GBPJPY",
    "EURGBP", "CADJPY", "CHFJPY", "AUDJPY", "EURNZD"
]
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01  # small size for $9 account
MIN_PROFIT_USD = 0.10
MAX_TRADES = 1
FORCE_MODE = False  # Don't force buy

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

for symbol in SYMBOLS:
    info = mt5.symbol_info(symbol)
    if info is None:
        raise RuntimeError(f"Symbol {symbol} not found")
    if not info.visible:
        mt5.symbol_select(symbol, True)

# === Helper: Get price data ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to get data for {symbol}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === Helper: Trade signal using multiple candlestick patterns ===
def get_trade_signal(df, symbol):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    engulf = talib.CDLENGULFING(open_, high, low, close)
    doji = talib.CDLDOJI(open_, high, low, close)
    morning_star = talib.CDLMORNINGSTAR(open_, high, low, close)
    hammer = talib.CDLHAMMER(open_, high, low, close)

    if len(rsi) == 0:
        return None

    last_rsi = rsi[-1]
    last_engulf = engulf[-1]
    last_doji = doji[-1]
    last_morning_star = morning_star[-1]
    last_hammer = hammer[-1]

    print(f"{symbol} - RSI: {last_rsi:.2f}, Engulf: {last_engulf}, Doji: {last_doji}, MorningStar: {last_morning_star}, Hammer: {last_hammer}")

    # === Buy conditions ===
    if (last_engulf > 0 or last_morning_star > 0 or last_hammer > 0) and last_rsi < 45:
        return "buy"

    # === Sell conditions ===
    if (last_engulf < 0 or last_doji < 0) and last_rsi > 60:
        return "sell"

    return None

# === Helper: Open trade ===
def open_trade(symbol, direction):
    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "profit_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"✅ OPENED {direction.upper()} for {symbol} at {price}")
    else:
        print(f"❌ Order failed for {symbol}: {result.retcode}")
        print(result._asdict())

# === Helper: Close trade if profitable ===
def check_and_close():
    for symbol in SYMBOLS:
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            pos = positions[0]
            profit = pos.profit
            print(f"{symbol} position PnL: ${profit:.2f}")
            if profit >= MIN_PROFIT_USD:
                close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask
                close_req = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": close_type,
                    "position": pos.ticket,
                    "price": price,
                    "deviation": 20,
                    "magic": 10001,
                    "comment": "close_profit",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(close_req)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ CLOSED {symbol} with profit ${profit:.2f}")
                else:
                    print(f"❌ Close failed for {symbol}: {result.retcode}")

# === MAIN LOOP ===
print("🚀 Running multi-pattern, profit-only bot...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        if len(mt5.positions_get() or []) >= MAX_TRADES:
            print("⛔ Trade limit reached. Waiting...")
            time.sleep(60)
            continue

        for symbol in SYMBOLS:
            if not any(pos.symbol == symbol for pos in mt5.positions_get() or []):
                df = get_data(symbol, TIMEFRAME, 100)
                signal = get_trade_signal(df, symbol)
                if signal:
                    open_trade(symbol, signal)

        time.sleep(60)
except KeyboardInterrupt:
    print("🛑 Bot manually stopped.")
finally:
    mt5.shutdown()


In [ ]:
uhaA
